In [18]:
import pandas as pd
import urllib
import os
from colorthief import ColorThief

In [3]:
data = pd.read_csv("yt8m_sample.csv")

In [4]:
data.columns.values

array(['Unnamed: 0', 'etag', 'id', 'kind', 'snippet.categoryId',
       'snippet.channelId', 'snippet.channelTitle',
       'snippet.defaultAudioLanguage', 'snippet.description',
       'snippet.liveBroadcastContent', 'snippet.localized.description',
       'snippet.localized.title', 'snippet.publishedAt', 'snippet.tags',
       'snippet.thumbnails.default.height',
       'snippet.thumbnails.default.url',
       'snippet.thumbnails.default.width',
       'snippet.thumbnails.high.height', 'snippet.thumbnails.high.url',
       'snippet.thumbnails.high.width', 'snippet.thumbnails.maxres.height',
       'snippet.thumbnails.maxres.url', 'snippet.thumbnails.maxres.width',
       'snippet.thumbnails.medium.height', 'snippet.thumbnails.medium.url',
       'snippet.thumbnails.medium.width',
       'snippet.thumbnails.standard.height',
       'snippet.thumbnails.standard.url',
       'snippet.thumbnails.standard.width', 'snippet.title',
       'statistics.commentCount', 'statistics.dislikeCount'

In [1]:
def download_thumbnail(url, id, folder="./thumbnails/"):  
    urllib.urlretrieve(url, "{}{}.jpg".format(folder, id))

In [17]:
for index, url, id in data[["snippet.thumbnails.default.url", "id"]].itertuples():
    download_thumbnail(url, id)

In [40]:
def get_color(id):
    path = "./thumbnails/{}.jpg".format(id)
    if not os.path.isfile(path):
        return None
    try:
        color_thief = ColorThief(path)
        # get the dominant color
        dominant_color = [str(c) for c in color_thief.get_color(quality=1)]
        return "rgb({})".format(",".join(dominant_color))
    except:
        return None

In [41]:
get_color("ZqGLXispP08")

'rgb(5,11,5)'

In [42]:
colors = [get_color(id) for id in data['id']]

In [43]:
data["dominant_color"] = colors

In [44]:
data_non_missing = data[data['dominant_color'] != None]

In [46]:
data_non_missing.to_json("./yt8m_sample_withcolor.json")

In [49]:
data_non_missing[
    ["id", 
     "statistics.viewCount", 
     "statistics.likeCount",
     "statistics.dislikeCount",
     "statistics.favoriteCount",
     "statistics.commentCount",
     "dominant_color"
    ]
].dropna().to_json("./yt8m_sample_stats.json")